# Algoritmo RSA

Suponiendo que Bob se quiere comuniar con Mary. Ambos deben generar su par de claves pública y privada.
Para esto eligen $p$ y $q$ que son primos aleatorios.

Para esto necesitamos una prueba rápida de primalidad. El algoritmo determinístico es demasiado lento ya que requiere hacer una búsqueda exhaustiva de los posibles divisores de un número.

En su lugar utilizaremos una prueba estocástica llamada Test de Miller Rabin.

De forma aleatoria generamos números aleatorios de un tamaño determinado y los sometemos a la prueba de Miller Rabin hasta obtener un número aleatorio primo.

In [ ]:
import random

def esPrimo(n): #no sirve
  divisor = 2
  while divisor < n/2:
    if (n % divisor == 0):
      return False
    divisor += 1
  print(divisor)
  return True

def rabinMiller(num):
    '''Comprueba si un numero num es probablemente primo'''
    if num%2 == 0:
        return False
    s = num - 1
    t = 0
    while s % 2 == 0:
        s = s // 2
        t += 1
   
    for pruebas in range(5):
        a = random.randrange(2, num - 1)
        v = pow(a,s,num)

        if v != 1:
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v**2)%num
    return True

def getPrimo(tam):
    while(True):
        num=random.randint(2**tam,2**(tam+1))
        if(rabinMiller(num)):
            return num 

Mary calcula su par de claves pública y privada, del orden de $2^{16}$. Las claves del algoritmo RSA seguras con del orden de $2^{2048}$ o más. Para este ejemplo vamos a trabajar con números relativamente pequeños del orden de $2^{16}$

In [ ]:
bits = 1024
p = getPrimo(bits)
q = getPrimo(bits)

Luego calculamos $z = pq$ y $ф=(p-1)(q-1)$

In [ ]:
z = p*q
phi = (p - 1) * (q - 1)

y luego elegimos un $n$ que es coprimo con ф, en este ejemplo lo elegimos primo que obviamente será coprimo con cualquier otro número. Si elegimos e primo se verificará que $n\ mod\ ф = 1$. Por lo que e tiene un inverso multiplicativo $s$

In [ ]:
def modinv(n, phi):
    '''calcula el inverso del modulo s de dos numeros n y phi'''
    def egcd(n, b):
        if n == 0:
            return (b, 0, 1)
        g, y, x = egcd(b%n,n)
        return (g, x - (b//n) * y, y)
    g, x, y = egcd(n, phi)
    if g != 1:
        raise Exception('No hay inverso del módulo')
    return x%phi

n = getPrimo(bits)
s = modinv(n,phi)

Ya tenemos las claves pública $(z,n)$ y privada $(z,s)$ de Mary 

In [ ]:
clavePublica = (z,n)
clavePrivada = (z,s)

Si Bob quiere enviar un mensaje M debe utilizar la calve pública de Mary. Suponiendo que el mensaje $a = 1000$. considerando que $0<a<z$

In [ ]:
a = 32476740775604083

Bob cifra el mensaje utilizando la clave pública de mary, y obtiene el mensaje cifrado $c = a^n mod\ z$

In [ ]:
c = pow(a,n,z)   #mensaje cifrado
c

20239363598151611374833551672086775454293470192524790002833876609220128860663712720239675831862249118951894828692916744799747699122061573654932824361478162109698626894814309269026391789869720875416509474703848040731814490615004400171143969451723786207594345758488604215267419689456994523408162372318169191211361517833823381228096362370774115823780751597734287198374773225815062186666580432474634549462741711678080588318928769907921687859952787098138046293404744279717740810974236132104314459892454280595000149633760245444398366049339608917932865050101108994897295771322820997506505078718318819374750869507221233454729

Luego envía a Mary el mensaje cifrado $c$

Cuando Mary recibe el mensaje cifrado $c$, lo descifra utilizando su propia clave privada $(z,s)$. Y obtiene el mensaje en claro $a = c^s\ mod \ z $.

In [ ]:
a = pow(c,s,z)   #mensaje descifrado
a

32476740775604083


## Ejercicio
Genere un par de claves pública y privada pública $(z,n)$ y privada $(z,s)$, luego guárdelas en un archivo y comparta su clave pública con otra persona.

In [ ]:
print(clavePublica)
print(clavePrivada)

(60234449989569166859209617906007107864337168385782895954574595185030277973296738972436424444511233154727383483498968555635492146414224611196631982474499194178715303293843975229116286746626606239565019508611398464294760502919226237229560109694721424410022860966618774533846901564722200060432580012916723015727636348169048018507447890865900338376180030257151955897002083663562676144128427278198383396691050011058085024033220053496022813795066183818726285546783713594859206369980234671199034650332071849173527382712923171987565896038282751003555127148818611318607784770749811527961755084245407672806364536329785068772819, 252817027195019717131188058480132765501045597423091252888179840366172735024880600921599983128605090850752559774652185592985206555908003643686670812738110332120616738040135753733784615198247348031052318131376454527621621418070002429500871587086389476673654348440701374693581840918166542669612222221643755649123)
(60234449989569166859209617906007107864337168385782895954574595185030

In [ ]:
import pickle
with open("soria_pub.rsa", "wb") as f:
  pickle.dump(clavePublica, f)

In [ ]:
with open("soria_pri.rsa", "wb") as f:
  pickle.dump(clavePrivada, f)